In [1]:
import pandas as pd

In [59]:
import pandas as pd

# Cargar los archivos CSV
API_merge = pd.read_csv('../data/API_data_Cleaned.csv')
db_merge = pd.read_csv('../data/us_accidents_cleaned.csv')

# Convertir las columnas de fecha a formato datetime
API_merge['crash_date'] = pd.to_datetime(API_merge['crash_date']).dt.date  # Usar solo la fecha
db_merge['start_time'] = pd.to_datetime(db_merge['start_time']).dt.date  # Usar solo la fecha

# Filtrar ambos conjuntos de datos por 'New York'
api_data_ny = API_merge[API_merge['city'] == 'New York']
us_accidents_ny = db_merge[db_merge['city'] == 'New York']

# Hacer un merge por la fecha (combinación interna)
merged_df = pd.merge(api_data_ny, us_accidents_ny, left_on='crash_date', right_on='start_time', how='inner')

merged_df = merged_df.drop(columns=['city_x','city_y'])

merged_df['city'] = "New York"

# Mover la columna 'id' al principio
cols = ['city'] + [col for col in merged_df.columns if col != 'city']
merged_df = merged_df[cols]


In [60]:
# Comprobar la cantidad de filas después del merge
merged_count = merged_df.shape[0]
print(f"Cantidad de filas después del merge: {merged_count}")

print(f"Null values: \n{merged_df.isnull().sum()}\n")

# Guardar el resultado del merge en un archivo CSV
merged_df.to_csv('../data/merged_data.csv', index=False, encoding='utf-8')

Cantidad de filas después del merge: 48001
Null values: 
city                             0
crash_date                       0
crash_time                       0
on_street_name                   0
off_street_name                  0
number_of_persons_injured        0
number_of_persons_killed         0
number_of_pedestrians_injured    0
number_of_pedestrians_killed     0
number_of_cyclist_injured        0
number_of_cyclist_killed         0
number_of_motorist_injured       0
number_of_motorist_killed        0
contributing_factor_vehicle_1    0
contributing_factor_vehicle_2    0
collision_id                     0
vehicle_type_code1               0
vehicle_type_code2               0
borough                          0
zip_code                         0
latitude                         0
longitude                        0
location                         0
severity                         0
start_time                       0
end_time                         0
start_lat                        